**Imports**

In [13]:
library(tidyverse)
library(emmeans)

options(pillar.sigfig = 4, digits=6)

# Compare familiarization-stage performance between groups

In [9]:
f <- function() { 
    # Prepare data
    df <- tbl_df(read.csv('data/clean_data.csv')) %>%
      dplyr::filter(stage=='train') %>%
      dplyr::select(sid,group,activity,correct)
    df$correct <- as.numeric(df$correct=='True')
    df$group <- as.factor(df$group)
    df$sid <- as.factor(df$sid)
    df$group <- dplyr::recode(df$group, '0'='IG', '1'='EG')

    # Calculate PC per task per subject
    df <- group_by(df, sid , activity) %>% dplyr::summarize(group = head(group, 1), pc = mean(correct))

    # Perform mixed ANOVA
    AOV_results <- aov(pc ~ group * activity + Error(sid), df)
    print(summary(AOV_results))
    
    # Post hoc analysis
    linmod <- lm(pc ~ group * activity, data=df, contrasts=list(group=contr.treatment(2), activity=contr.treatment(4)))
    posthoc <- emmeans(linmod, 'activity', adjust = "tukey")
    print(pairs(posthoc))
}

f()


Error: sid
           Df Sum Sq Mean Sq F value Pr(>F)
group       1  0.066 0.06566   2.256  0.134
Residuals 328  9.547 0.02911               

Error: Within
                Df Sum Sq Mean Sq F value Pr(>F)    
activity         3  9.827   3.276 141.098 <2e-16 ***
group:activity   3  0.110   0.037   1.574  0.194    
Residuals      984 22.844   0.023                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


NOTE: Results may be misleading due to involvement in interactions


 contrast estimate     SE   df t.ratio p.value
 A1 - A2    0.0664 0.0123 1312  5.418  <.0001 
 A1 - A3    0.1473 0.0123 1312 12.015  <.0001 
 A1 - A4    0.2299 0.0123 1312 18.750  <.0001 
 A2 - A3    0.0809 0.0123 1312  6.597  <.0001 
 A2 - A4    0.1634 0.0123 1312 13.332  <.0001 
 A3 - A4    0.0826 0.0123 1312  6.736  <.0001 

Results are averaged over the levels of: group 
P value adjustment: tukey method for comparing a family of 4 estimates 


# Compare activity preferences betwen groups

In [15]:
f <- function() { 
    # Prepare data
    df <- read.csv('data/clean_data.csv') %>%
      dplyr::filter(trial<=60+250) %>%
      dplyr::select(sid,group,trial,activity) %>% 
      dplyr::group_by(group,sid,activity) %>% 
      dplyr::summarize(count=n()) %>%
      dplyr::mutate(relt=(count-15)/250)
    df$group <- dplyr::recode(factor(df$group), '0'='IG', '1'='EG')
    df$sid <- as.factor(df$sid)

    # Print stats
    (dplyr::group_by(df, group, activity) %>% dplyr::summarize(mean=mean(relt), std=sd(relt)))

    # Fit a linear mixed effects model
    df <- within(df, activity <- relevel(activity, ref='A1'))
    linmod <- lm(
      relt ~ group * activity, 
      data = df, 
      contrasts = list(group=contr.treatment(2), activity=contr.sum(4))
    )
    print(summary(linmod), digits=6)

    # Perform mixed ANOVA
    AOV_results <- aov(relt ~ group * activity + Error(sid), df)
    print(summary(AOV_results), digits=6)
}

f()


Call:
lm(formula = relt ~ group * activity, data = df, contrasts = list(group = contr.treatment(2), 
    activity = contr.sum(4)))

Residuals:
       Min         1Q     Median         3Q        Max 
-0.3882955 -0.1256250 -0.0409740  0.0981161  0.7143864 

Coefficients:
                     Estimate   Std. Error  t value   Pr(>|t|)    
(Intercept)       2.50000e-01  7.54760e-03 33.12313 < 2.22e-16 ***
group2            1.18405e-16  1.03350e-02  0.00000  1.0000000    
activity1         3.80779e-02  1.30728e-02  2.91276  0.0036432 ** 
activity2        -1.69870e-02  1.30728e-02 -1.29941  0.1940299    
activity3        -2.10649e-02  1.30728e-02 -1.61135  0.1073432    
group2:activity1 -1.26464e-01  1.79007e-02 -7.06477 2.6038e-12 ***
group2:activity2 -3.53539e-02  1.79007e-02 -1.97500  0.0484773 *  
group2:activity3  2.34968e-02  1.79007e-02  1.31262  0.1895416    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.187326 on 1312 degrees of freed

# Fitted coefficients and difficulty preferences

In [2]:
f <- function() { 
    # Load parameter data
    df1 <- tbl_df(read.csv('data/model_results/param_fits_clean.csv')) %>%
      dplyr::filter(vars=='rpc,rlp', nam > 0) %>%
      dplyr::mutate(rpc_norm = rpc/sqrt(rpc^2+rlp^2), rlp_norm = rlp/sqrt(rpc^2+rlp^2)) #%>%
      # dplyr::mutate(wPC = (wPC - mean(wPC))/sd(wPC),
      #               wLP = (wLP - mean(wLP))/sd(wLP))

    # Load RelT data
    df2 <- tbl_df(read.csv('data/model_data.csv')) %>% 
      dplyr::select(sid, trial, abst1, abst2, abst3, abst4) %>%
      dplyr::mutate(A4minusA3 = abst4-abst3, A3minusA1 = abst3-abst1) %>%
      dplyr::filter(trial==250)

    # Join datasets
    df <- dplyr::inner_join(df1, df2, by='sid')
    remove(df1, df2)

    # Convert categorical variables to factors
    df <- dplyr::mutate_at(df, vars(sid, group, nam), funs(factor))

    # Set colors
    df$group <- recode(df$group, '0' = 'IG', '1' = 'EG')

#     # Fit linear models
#     df <- within(df, group <- relevel(group, ref = 'IG'))
#     lm_ <- lm(A3minusA1 ~ group*(rpc_norm+rlp_norm), data=df)
#     print(summary(lm_))
#     lm_ <- lm(A4minusA3 ~ group*(rpc_norm+rlp_norm), data=df)
#     print(summary(lm_))
    # Fit linear models
    df <- within(df, group <- relevel(group, ref = 'IG'))
    lm_ <- lm(rpc_norm ~ group*(A3minusA1+A4minusA3), data=df)
    print(summary(lm_))
    lm_ <- lm(rlp_norm ~ group*(A3minusA1+A4minusA3), data=df)
    print(summary(lm_))
}

f()

Warning message:
“funs() is soft deprecated as of dplyr 0.8.0
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once per session.”


Call:
lm(formula = rpc_norm ~ group * (A3minusA1 + A4minusA3), data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.24241 -0.33406 -0.04356  0.33188  1.52479 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)        0.0976458  0.0408726   2.389   0.0175 *  
groupEG           -0.0896801  0.0611854  -1.466   0.1437    
A3minusA1         -0.0055262  0.0004780 -11.560   <2e-16 ***
A4minusA3         -0.0057868  0.0006642  -8.712   <2e-16 ***
groupEG:A3minusA1 -0.0015969  0.0007961  -2.006   0.0457 *  
groupEG:A4minusA3  0.0008581  0.0008340   1.029   0.3043    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4903 on 314 degrees of freedom
Multiple R-squared:  0.5802,	Adjusted R-squared:  0.5735 
F-statistic:  86.8 on 5 and 314 DF,  p-value: < 2.2e-16


Call:
lm(formula = rlp_norm ~ group * (A3minusA1 + A4minusA3), data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.34